In [99]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [100]:
matches = pd.read_csv("assets/2023-2024.csv")

In [101]:
matches["Date Time (US Eastern)"] = pd.to_datetime(matches["Date Time (US Eastern)"])

""" matches["Date (US Eastern)"] = pd.to_datetime(matches["Date (US Eastern)"])
matches["Time (US Eastern)"] = pd.to_datetime(matches["Time (US Eastern)"]) """

/var/folders/cn/j21vzthj6ds5gjqdtwlgj4f00000gn/T/ipykernel_1068/1605438902.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  matches["Date Time (US Eastern)"] = pd.to_datetime(matches["Date Time (US Eastern)"])


' matches["Date (US Eastern)"] = pd.to_datetime(matches["Date (US Eastern)"])\nmatches["Time (US Eastern)"] = pd.to_datetime(matches["Time (US Eastern)"]) '

In [102]:
#advantages and disadvantages to consider when teams play against each other

matches["homeAway_code"] = matches["homeAway"].astype("category").cat.codes

matches["opponent_code"] = np.where(
    matches["Team"] == matches["Home Team"],
    matches["Away Team"].astype("category").cat.codes,
    matches["Home Team"].astype("category").cat.codes
)

matches["hour"] = matches["Date Time (US Eastern)"].dt.hour.astype("Int64")
matches["day_code"] = matches["Date Time (US Eastern)"].dt.dayofweek
matches["date"] = matches["Date Time (US Eastern)"].dt.date
""" 
info to train the model
2. On target shots
3. Goal (expected goals)
4. Possession 
5. Pass completsion 
"""

' \ninfo to train the model\n2. On target shots\n3. Goal (expected goals)\n4. Possession \n5. Pass completsion \n'

In [103]:
#determine the match result
matches["target"] = np.where(
    matches["Home Goal"] == matches["Away Goal"], 2,  
    np.where(
        (matches["Home Goal"] > matches["Away Goal"]) & (matches["Team"] == matches["Home Team"]), 1,  
        np.where(
            (matches["Home Goal"] > matches["Away Goal"]) & (matches["Team"] != matches["Home Team"]), 0,  
            np.where(
                (matches["Home Goal"] < matches["Away Goal"]) & (matches["Team"] == matches["Away Team"]), 1,  
                0  
            )
        )
    )
)

In [104]:
#began using machine learning 
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [105]:
predictors = ["homeAway_code", "opponent_code", "hour", "day_code"]

In [106]:
train = matches[matches["Date Time (US Eastern)"] <= '2024-01-01']

In [107]:
test = matches[matches["Date Time (US Eastern)"] > '2024-01-01']

In [108]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [109]:
preds = rf.predict(test[predictors])

In [110]:
accuracy = accuracy_score(test["target"], preds)
accuracy

0.4076086956521739

In [111]:
combine = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [112]:
pd.crosstab(index=combine["actual"], columns=combine["prediction"])

prediction,0,1,2
actual,,,
0,64,62,12
1,51,81,6
2,41,46,5


In [113]:
precision_score(test["target"], preds, average='macro', zero_division=0)

0.35207304772522163

In [114]:
grouped_matches = matches.groupby("Team")
#group = grouped_matches.get_group("Liverpool")

In [143]:
""" 
This function uses the stats from the previous 3 games 
to determine the team's condition and predict the next game. 
----basically getting the averge of the team stats---
"""
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Date Time (US Eastern)")
    rolling_stats = group[cols].shift(1).rolling(window=3).mean()
    for col, new_col in zip(cols, new_cols):
            group[new_col] = rolling_stats[col]
    group = group.dropna(subset=new_cols)
    return group

In [140]:
columns = [
    "totalShots", "shotsOnTarget", 
    "possessionPct", "wonCorners", 
    "accuratePasses", "totalPasses", 
    "effectiveTackles", "totalTackles"]
new_columns = [f"{c}_rolling" for c in columns]

In [141]:
""" rolling_averages(group, columns, new_columns) """
matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, columns, new_columns))
matches_rolling.droplevel("Team")
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling

/var/folders/cn/j21vzthj6ds5gjqdtwlgj4f00000gn/T/ipykernel_1068/2449593900.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, columns, new_columns))


,Team,Date Time (US Eastern),Fixture,Home Team,Home Goal,Away Goal,Away Team,homeAway,foulsCommitted,yellowCards,...,date,target,totalShots_rolling,shotsOnTarget_rolling,possessionPct_rolling,wonCorners_rolling,accuratePasses_rolling,totalPasses_rolling,effectiveTackles_rolling,totalTackles_rolling
0,AFC Bournemouth,2023-09-02 10:00:00,AFC Bournemouth at Brentford,Brentford,2.0,2.0,AFC Bournemouth,away,13.0,2.0,...,2023-09-02,2,12.666667,4.333333,47.000000,6.000000,323.666667,411.000000,8.333333,17.333333
1,AFC Bournemouth,2023-09-17 09:00:00,Chelsea at AFC Bournemouth,AFC Bournemouth,0.0,0.0,Chelsea,home,15.0,1.0,...,2023-09-17,2,12.000000,4.333333,39.000000,3.333333,257.333333,342.000000,9.666667,19.000000
2,AFC Bournemouth,2023-09-24 09:00:00,AFC Bournemouth at Brighton & Hove Albion,Brighton & Hove Albion,3.0,1.0,AFC Bournemouth,away,17.0,3.0,...,2023-09-24,0,12.000000,4.000000,39.000000,3.000000,260.666667,337.666667,9.333333,17.666667
3,AFC Bournemouth,2023-09-30 10:00:00,Arsenal at AFC Bournemouth,AFC Bournemouth,0.0,4.0,Arsenal,home,11.0,0.0,...,2023-09-30,0,12.333333,4.333333,37.666667,2.333333,239.666667,315.666667,10.000000,19.666667
4,AFC Bournemouth,2023-10-07 10:00:00,AFC Bournemouth at Everton,Everton,3.0,0.0,AFC Bournemouth,away,15.0,2.0,...,2023-10-07,0,11.000000,3.000000,38.666667,3.666667,270.666667,342.000000,9.666667,17.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Wolverhampton Wanderers,2024-04-24 14:45:00,AFC Bournemouth at Wolverhampton Wanderers,Wolverhampton Wanderers,0.0,1.0,AFC Bournemouth,home,10.0,4.0,...,2024-04-24,0,10.000000,4.666667,52.666667,2.000000,450.666667,528.333333,14.666667,24.000000
696,Wolverhampton Wanderers,2024-04-27 10:00:00,Luton Town at Wolverhampton Wanderers,Wolverhampton Wanderers,2.0,1.0,Luton Town,home,14.0,3.0,...,2024-04-27,1,10.333333,3.666667,54.333333,3.333333,423.000000,508.333333,11.000000,18.000000
697,Wolverhampton Wanderers,2024-05-04 12:30:00,Wolverhampton Wanderers at Manchester City,Manchester City,5.0,1.0,Wolverhampton Wanderers,away,17.0,4.0,...,2024-05-04,0,11.000000,4.000000,55.666667,4.333333,450.333333,537.666667,11.666667,20.666667
698,Wolverhampton Wanderers,2024-05-11 10:00:00,Crystal Palace at Wolverhampton Wanderers,Wolverhampton Wanderers,1.0,3.0,Crystal Palace,home,13.0,3.0,...,2024-05-11,0,10.000000,3.333333,54.333333,4.000000,451.333333,534.000000,11.333333,19.666667
